## Load sub

The videos are in the order of Experiment_id, so not in the order of presentation. This means the first video is the same for each participant.

In [1]:
import pickle
import numpy as np
import gc  # Garbage collector

!pip install mrmr-selection
!pip install ordpy

   ---------------------------------------- 0.0/31.7 MB ? eta -:--:--
    --------------------------------------- 0.8/31.7 MB 8.5 MB/s eta 0:00:04
   --- ------------------------------------ 2.9/31.7 MB 8.0 MB/s eta 0:00:04
   ----- ---------------------------------- 4.7/31.7 MB 8.2 MB/s eta 0:00:04
   -------- ------------------------------- 6.6/31.7 MB 8.4 MB/s eta 0:00:04
   ---------- ----------------------------- 8.4/31.7 MB 8.5 MB/s eta 0:00:03
   ------------ --------------------------- 10.2/31.7 MB 8.5 MB/s eta 0:00:03
   --------------- ------------------------ 12.1/31.7 MB 8.5 MB/s eta 0:00:03
   ----------------- ---------------------- 13.9/31.7 MB 8.6 MB/s eta 0:00:03
   ------------------- -------------------- 15.5/31.7 MB 8.3 MB/s eta 0:00:02
   --------------------- ------------------ 17.0/31.7 MB 8.4 MB/s eta 0:00:02
   ----------------------- ---------------- 18.9/31.7 MB 8.4 MB/s eta 0:00:02
   -------------------------- ------------- 20.7/31.7 MB 8.4 MB/s eta 0:00:02

In [ ]:
participants = 32
subjects = {'data': [], 'labels': []}
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Stress\\deap-dataset\\'

# Read and collect data
for i in range(1, participants + 1):
    file_name = prefix_path + f"data_preprocessed_python\\s{'0' if i < 10 else ''}{i}.dat"
    with open(file_name, 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
        for key in subjects:
            subjects[key].append(subject[key])

        del subject

# Merge data and reshape
for key in subjects:
    subjects[key] = np.concatenate(subjects[key], axis=0)

print(subjects['data'].shape)
print(subjects['labels'].shape)

(1280, 40, 8064)
(1280, 4)


In [5]:
# Use 2 classifiers valence and arousal
subjects['labels'] = subjects['labels'][:, :2]

# Use threshold 5 to assign binary label to each classifier
subjects['labels'][:, 0] = (subjects['labels'][:, 0] >= 5).astype(int)
subjects['labels'][:, 1] = (subjects['labels'][:, 1] >= 5).astype(int)

print(set(subjects['labels'][:, 0]))
print(set(subjects['labels'][:, 1]))

{0.0, 1.0}
{0.0, 1.0}


## Split Train Test Dataset

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    subjects['data'], subjects['labels'], test_size=test_size, random_state=random_state
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

del subjects
gc.collect()  # Free up memory

(1024, 40, 8064) (256, 40, 8064) (1024, 2) (256, 2)


0

## Feature extraction


### Wavelet transform

In [7]:
# Wavelet transform
import pywt
from tqdm import tqdm

# Sampling frequency:
fs = 128

# Select scales and wavelets
scales = np.arange(1, 129)  # Example: 129 scale values
wavelet = 'morl'

def CWT_for_sample(data):
    cwt_output = []
    # Calculate CWT for each sample and each channel
    for i in tqdm(range(data.shape[0])):  # 1024 samples
        cwt_channels = []
        for j in range(data.shape[1]):  # 40 channels
            coeffs, freqs = pywt.cwt(data[i, j], scales, wavelet) # coeffs are of the form (len(scales), 8064)
            energy_scales = np.sum(np.abs(coeffs)**2, axis=1)  # (len(scales),)
            cwt_channels.append(energy_scales)
        cwt_output.append(cwt_channels)
    
    # Convert output to numpy array
    cwt_output = np.array(cwt_output)  # Size: (1024, 40, len(scales))

    return cwt_output
    
# cwt_train = CWT_for_sample(X_train) # Size: (1024, 40, len(scales))
# cwt_test = CWT_for_sample(X_test) # Size: (256, 40, len(scales))

# print(cwt_train.shape, cwt_test.shape)

# # Save file name
# prefix_output_path = '/kaggle/working/'
# filename = "cwt_data.pkl"

# # Save data
# with open(filename, 'wb') as file:
#     pickle.dump({'train': cwt_train, 'test': cwt_test}, file)

# print(f"Data has been saved to file {filename}")


In [24]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Stress\\deap-dataset\\'
filename = "cwt_data.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)

# Data retrieval
cwt_train_loaded = data['train']
cwt_test_loaded = data['test']

print("Size cwt_train:", cwt_train_loaded.shape)
print("Size cwt_test:", cwt_test_loaded.shape)


Size cwt_train: (1024, 40, 128)
Size cwt_test: (256, 40, 128)


### Nonlinear Feature Analyses: Permutation Entropy

In [11]:
import ordpy

def split_array(arr, num_epochs):
    
    len_arr = len(arr)
    len_epochs = round(len_arr / num_epochs)
    
    splits_arr = [arr[i * len_epochs: len_arr if i + 1 == num_epochs else (i + 1) * len_epochs] for i in range(num_epochs)]

    return splits_arr

def permutation_entropy_eeg(data, num_epochs=8):
    B, C, T = data.shape
    pe = np.zeros((B, C, num_epochs))
    
    for b in tqdm(range(B)):
        for c in range(C):
            splits_arr = split_array(data[b, c], num_epochs)
            for i in range(num_epochs):
                pe[b, c, i] = ordpy.permutation_entropy(splits_arr[i])

    return pe

# pe_train = permutation_entropy_eeg(X_train)#1024x40
# pe_test = permutation_entropy_eeg(X_test)#256x40

# # Save file name
# prefix_output_path = '/kaggle/working/'
# filename = "pe_data.pkl"

# # Save data
# with open(filename, 'wb') as file:
#    pickle.dump({'train': pe_train, 'test': pe_test}, file)

# print(f"Data has been saved to file {filename}")

In [25]:
# # Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Stress\\deap-dataset\\'
filename = "pe_data.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)

# Data retrieval
pe_train_loaded = data['train']
pe_test_loaded = data['test']

print("Size pe_train:", pe_train_loaded.shape)
print("Size pe_test:", pe_test_loaded.shape)

Size pe_train: (1024, 40, 8)
Size pe_test: (256, 40, 8)


## Feature Selection

In [14]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def pca_data(data, k):
    
    # Reshape the data into (n_samples, n_features) to prepare for PCA
    n_samples, n_channels, n_features = data.shape
    data_reshaped = data.reshape(n_samples * n_channels, n_features)
    
    # Data Normalization
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_reshaped)
    
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=k)
    data_pca = pca.fit_transform(data_scaled)
    
    # Reshape back to (1024, 40, k)
    data_pca_reshaped = data_pca.reshape(n_samples, n_channels, k)
    
    return data_pca_reshaped

# pca_train = pca_data(X_train, k=128)
# pca_test = pca_data(X_test, k=128)

# # Save file name
# prefix_output_path = '/kaggle/working/'
# filename = "pca_data.pkl"

# # Save data
# with open(filename, 'wb') as file:
#    pickle.dump({'train': pca_train, 'test': pca_test}, file)

# print(f"Data has been saved to file {filename}")


In [26]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Stress\\deap-dataset\\'
filename = "pca_data.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)

# Data retrieval
pca_train_loaded = data['train']
pca_test_loaded = data['test']

print("Size pca_train:", pca_train_loaded.shape)
print("Size pca_test:", pca_test_loaded.shape)

Size pca_train: (1024, 40, 128)
Size pca_test: (256, 40, 128)


In [16]:
from mrmr import mrmr_classif
import pandas as pd

def feature_selector(num_features, X, y, classif_obj): # giữ lại kích thước chiều cuối cùng
    flatted_X = X.reshape(-1, X.shape[-1])

    flatted_X = pd.DataFrame(flatted_X)

    flatted_y_new = y
    if len(y.shape) == 2:
        y_new = np.repeat(y[:, np.newaxis, :], X.shape[1], axis=1)
        flatted_y_new = y_new.reshape(-1, y_new.shape[-1])
        
    flatted_y_new = flatted_y_new[:,classif_obj]

    selected_features = mrmr_classif(X=flatted_X, y=flatted_y_new, K=num_features)

    return selected_features

# f = feature_selector(64, cwt_train_loaded, y_train, 0)
# feature_selections = {
#     'PCA': {
#         'Valence': feature_selector(64, pca_train_loaded, y_train, 0), #1024x40x8064
#         'Arousal': feature_selector(64, pca_train_loaded, y_train, 1) #1024x40x8064
#     },
#     'WT': {
#         'Valence': feature_selector(64, cwt_train_loaded, y_train, 0), #1024x40x128
#         'Arousal': feature_selector(64, cwt_train_loaded, y_train, 1) #1024x40x128
#     }
# }

# # Save file name
# prefix_output_path = '/kaggle/working/'
# filename = "feature_selections.pkl"

# # Save data
# with open(filename, 'wb') as file:
#     pickle.dump(feature_selections, file)

# print(f"Data has been saved to file {filename}")

In [17]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    feature_selection_loaded = pickle.load(file)

print(feature_selection_loaded)

{'PCA': {'Valence': [62, 96, 53, 87, 15, 66, 83, 11, 113, 46, 125, 39, 38, 116, 6, 68, 56, 57, 36, 123, 126, 120, 69, 10, 51, 9, 119, 35, 99, 94, 43, 40, 42, 52, 90, 59, 16, 105, 118, 64, 27, 49, 63, 124, 101, 17, 98, 86, 5, 111, 81, 80, 33, 109, 45, 8, 61, 54, 1, 50, 112, 37, 29, 121], 'Arousal': [98, 59, 94, 9, 72, 97, 47, 56, 21, 17, 71, 83, 50, 5, 2, 39, 69, 49, 107, 37, 48, 65, 114, 27, 38, 20, 42, 11, 8, 112, 80, 81, 84, 14, 0, 91, 120, 1, 40, 118, 123, 28, 12, 86, 46, 104, 4, 74, 108, 3, 125, 88, 122, 55, 106, 29, 115, 30, 113, 66, 87, 119, 23, 41]}, 'WT': {'Valence': [52, 51, 53, 54, 50, 55, 56, 49, 57, 58, 48, 59, 1, 60, 61, 62, 63, 47, 64, 5, 65, 6, 66, 4, 67, 7, 68, 46, 0, 69, 8, 3, 70, 9, 10, 71, 30, 2, 31, 29, 11, 32, 17, 72, 16, 12, 18, 45, 15, 28, 13, 73, 14, 33, 19, 74, 20, 27, 34, 75, 21, 26, 35, 44], 'Arousal': [64, 63, 65, 66, 62, 67, 61, 68, 60, 69, 70, 59, 71, 58, 72, 73, 33, 57, 32, 34, 74, 31, 75, 56, 35, 76, 30, 36, 77, 55, 78, 37, 29, 79, 54, 80, 38, 81, 28, 53

## Data synthesis

In [18]:
aggregated_data = [
    ['WT', 'YES', 'VALENCE', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_train), 
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_test)}],
    ['WT', 'YES', 'AROUSAL', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_train),
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_test)}],

    ['WT', 'NO', '*', {'TRAIN': (cwt_train_loaded, y_train), 
                       'TEST': (cwt_test_loaded, y_test)}],

    ['PE', 'NO', '*', {'TRAIN': (pe_train_loaded, y_train), 
                       'TEST': (pe_test_loaded, y_test)}],

    ['NONE', 'YES', 'VALENCE', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_test)}],
    ['NONE', 'YES', 'AROUSAL', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_test)}],

    ['NONE', 'NO', '*', {'TRAIN': (pca_train_loaded, y_train), 
                         'TEST': (pca_test_loaded, y_test)}],
]

## Model

In [19]:
#raw: 1024x40x8064 -> 
# feature extraction: 1024x40x128 -> 
# feature selection: 1024x5120 -> 1024xK

# select each extraction method, selection for each model -> get the best result of each model to compare with each other
# SVM, random forest: need to flatten input

In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def calculate_metric(metric, y_true, y_pred):
    """
    Calculates the value of a metric based on the input metric name.

    Args:
        metric (str): Metric name, one of the ['Acc', 'Sens', 'Spec', 'Prec', 'F-measure'].
        y_true (array-like): Actual label.
        y_pred (array-like): Predictive label.

    Returns:
        float: Value of selected metric.

    Raises:
        ValueError: If the metric is not in the supported list.
    """
    metric = metric.lower()  # Convert metrics to lowercase for easier handling

    if metric == 'acc':
        return accuracy_score(y_true, y_pred)
    elif metric == 'sens':  # Sensitivity (Recall)
        return recall_score(y_true, y_pred, average='binary')
    elif metric == 'spec':  # Specificity
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        return tn / (tn + fp) if (tn + fp) != 0 else 0.0
    elif metric == 'prec':  # Precision
        return precision_score(y_true, y_pred, average='binary')
    elif metric == 'f-measure':  # F1-Score
        return f1_score(y_true, y_pred, average='binary')
    else:
        raise ValueError(f"Metric '{metric}' not supported. Please select one of the ['Acc', 'Sens', 'Spec', 'Prec', 'F-measure'].")
    

def evaluate(model, name_model, classifi_obj, X, y):

    if name_model in ['svm', 'rf']:
        X = X.reshape(X.shape[0], -1)
        y_pred = model.predict(X)
        
    else:
        X = torch.as_tensor(X, dtype=torch.float32).to(DEVICE)
        if name_model == 'rnn':
            X = X.permute(0, 2, 1)
        y_pred = model(X)
        y_pred = (F.sigmoid(y_pred) >= 0.5).float().cpu().numpy()

    result = {
        metric: calculate_metric(
            metric, 
            y[:, classifi_obj],
            y_pred
        )
        for metric in ['Acc', 'Sens', 'Spec', 'Prec', 'F-measure']
    }

    return result

In [21]:
class EarlyStopping:
    def __init__(self, patience=5, delta=1e-3, save_path='checkpoint.pth'):
        """
        :param patience: number of waits before stopping (number of epochs where validation loss does not improve)
        :param delta: minimum change in validation loss to be considered an improvement
        :param save_path: path to save model when validation loss improves
        """
        self.patience = patience
        self.delta = delta
        self.save_path = save_path
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            # self.save_checkpoint(val_loss, model)
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            # self.save_checkpoint(val_loss, model)
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, val_loss, model):
        """Save model when validation loss improves"""
        torch.save(model.state_dict(), self.save_path)
        print(f"Validation loss decrease: {val_loss:.6f}, model saved.")

def setup_config_train_deep_model(train_data_X, train_data_y, test_data_X, test_data_y, name_model):
    batch_size = 64
    device=DEVICE
    
    # Create TensorDataset for train and test
    train_dataset = TensorDataset(torch.as_tensor(train_data_X, device=device), torch.as_tensor(train_data_y, device=device))
    test_dataset = TensorDataset(torch.as_tensor(test_data_X, device=device), torch.as_tensor(test_data_y, device=device))
    
    # Create DataLoader for train and test with batch_size=64 (or custom)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    num_epochs = 20
    optimizer = Adam
    lr = 1e-4
    prefix_output_path = '/kaggle/working/'

    config = {
        'train_dataloader': train_dataloader,
        'test_dataloader': test_dataloader,
        'num_epochs': num_epochs,
        'lr': lr,
        'optimizer': optimizer,
        'prefix_output_path': prefix_output_path,
        'name_model': name_model
    }

    return config
    
def train_deep_model(model, classifi_obj, **kwargs):
    name_model = kwargs.get('name_model')
    lr = kwargs.get('lr')
    optimizer = kwargs.get('optimizer')(model.parameters(), lr=lr)
    num_epochs = kwargs.get('num_epochs')
    
    scheduler = StepLR(optimizer, step_size=7, gamma=0.9)
    criterion = nn.BCEWithLogitsLoss()
    train_loss = []
    val_loss = []
    log = ""
    
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_dataloader = kwargs.get('train_dataloader')
        train_epoch_loss = []
        
        for i, (X_batch, y_batch) in enumerate(train_dataloader):
            optimizer.zero_grad()

            X_batch = X_batch.to(torch.float32)
            y_batch = y_batch.to(torch.float32)
            
            if name_model.lower() == 'rnn':
                X_batch = X_batch.permute(0,2,1)
                
            y_pred = model(X_batch)
            
            loss = criterion(y_pred, y_batch[:, classifi_obj])
            train_epoch_loss.append(loss.item())
            
            loss.backward()
            optimizer.step()

        scheduler.step()
        train_epoch_loss = np.mean(train_epoch_loss)
        train_loss.append(train_epoch_loss)

        # Calculate validation loss
        test_dataloader = kwargs.get('test_dataloader')
        model.eval()  # Switch to eval mode
        val_epoch_loss = []
        with torch.no_grad():
            for X_batch, y_batch in test_dataloader:
                
                X_batch = X_batch.to(torch.float32)
                y_batch = y_batch.to(torch.float32)

                if name_model.lower() == 'rnn':
                    X_batch = X_batch.permute(0,2,1)
                
                outputs = model(X_batch)
                
                loss = criterion(outputs, y_batch[:, classifi_obj])
                val_epoch_loss.append(loss.item())

        val_epoch_loss = np.mean(val_epoch_loss)
        val_loss.append(val_epoch_loss)

        prefix_output_path = kwargs.get('prefix_output_path')
                
        log += '\n' + f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}, lr: {scheduler.get_last_lr()}"
        
        early_stopping = EarlyStopping(val_loss, model, save_path=f'{prefix_output_path}{name_model}.pth')
        if early_stopping.early_stop:
            print("Early stopping!")
            # early_stopping.save_checkpoint(val_loss, model)
            break
            
    # Save data
    # with open(f'{prefix_output_path}log_{name_model}.txt', 'w') as file:
    #     file.write(log)

    return train_loss, val_loss, log
    # pass

def setup_config_train_ml_model(X_train, y_train, X_test, y_test, name_model):
    return {
        'X_train': X_train,
        'y_train': y_train,
        'name_model': name_model.lower()
    }

def train_ml_model(model, classifi_obj, **kwargs):
    name_model = kwargs.get('name_model').lower()
    X_train = kwargs.get('X_train')
    X_train = X_train.reshape(X_train.shape[0], -1)
    y = kwargs.get('y_train')[:, classifi_obj]
    model.fit(X_train, y)

def setup_config_train(X_train, y_train, X_test, y_test, name_model):
    name_model = name_model.lower()
    if name_model in ['svm', 'rf']:
        
        return setup_config_train_ml_model(X_train, y_train, X_test, y_test, name_model)

    else:

        return setup_config_train_deep_model(X_train, y_train, X_test, y_test, name_model)

def train_model(model, classifi_obj, **kwargs):
    if kwargs.get('name_model').lower() in ['svm', 'rf']:

        return train_ml_model(model, classifi_obj, **kwargs)

    else:

        return train_deep_model(model, classifi_obj, **kwargs)
    

In [22]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, rnn_type="LSTM", device=DEVICE):
        super(RNNModel, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Choose RNN type: LSTM, GRU, or vanilla RNN
        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, device=device)
        elif rnn_type == "GRU":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, device=device)
        else:
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, device=device)

        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, output_size, device=device)

    def forward(self, x):
        # Initialize hidden state and cell state (if using LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state

        if isinstance(self.rnn, nn.LSTM):
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state
            out, _ = self.rnn(x, (h0, c0))
        else:
            out, _ = self.rnn(x, h0)

        # Take the last output of the sequence
        out = self.fc(out[:, -1, :])

        out = out.squeeze()
        
        return out

class CNNModel(nn.Module):
    def __init__(self, in_channels, outs_channels, output_size, length_signal, device=DEVICE):
        super().__init__()

        channels = [in_channels] + outs_channels
        self.model = nn.Sequential(
            *[nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, device=device),
                nn.BatchNorm1d(out_channels, device=device),
                nn.ReLU()
            ) for in_channels, out_channels in zip(channels[:-1], channels[1:])]
        )
            
        self.fc = nn.Linear(outs_channels[-1] * length_signal, output_size, device=device) 

    def forward(self, x):
        x = self.model(x)

        x = x.view(x.size(0), -1)

        x = self.fc(x)

        x = x.squeeze()

        return x
    
def get_model(**kwargs):
    model = None
    name_model = kwargs.get('name_model').lower()
    
    device = kwargs.get('device', DEVICE)
    
    if name_model == 'svm':
        model = SVC(kernel='rbf')
        
    elif name_model == 'rf':
        model = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_leaf=5, random_state=42)
        
    elif name_model == 'rnn':
        # with torch.no_grad():
        #     dummy = torch.Tensor(1024, 40, 128).permute(0, 2, 1)# -> 1024, 128, 40
        input_features = kwargs.get('input_features', 40)
        hidden_size = kwargs.get('hidden_size', 128)
        num_layers = kwargs.get('num_layers', 4)
        output_size = kwargs.get('output_size', 1) # 1 output unit for binary classification
        length_signal = kwargs.get('length_signal', 128)
        
        model = RNNModel(
            input_size=input_features, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            output_size=output_size, 
            rnn_type="LSTM"
        )
        
    elif name_model == 'cnn':
        in_channels = kwargs.get('in_channels', 40)
        outs_channels = kwargs.get('outs_channels', [64,256,128])
        output_size = kwargs.get('output_size', 1)
        length_signal = kwargs.get('length_signal', 128)
        
        model = CNNModel(
            in_channels=in_channels,
            outs_channels=outs_channels,
            output_size=output_size,
            length_signal=length_signal,
            device=device
        )

    return model

In [18]:
# X_data_train, y_data_train = aggregated_data[0][-1]['TRAIN']
# X_data_test, y_data_test = aggregated_data[0][-1]['TEST']

# name_model = 'svm'
# classifi_obj = 0

# config = {'name_model': name_model, 'length_signal': 64}
# model = get_model(**config)

# config_train = setup_config_train(X_data_train, y_data_train, X_data_test, y_data_test, name_model)
# # print(config_train)
# info = train_model(model, classifi_obj, **config_train)
# print(info)

# print(evaluate(model, name_model, classifi_obj, X_data_test, y_data_test))
# # out = model(X)
# # print(out.shape)

In [23]:
models = []
evaluates = []
i = 0
for name_model in ['rf', 'svm', 'rnn', 'cnn']:
    for experiment in aggregated_data:
        data = experiment[-1]
        classifi_objs = [0] if experiment[2] == 'VALENCE' else [1] if experiment[2] == 'AROUSAL' else [0,1]
        (X_train_data, y_train_data), (X_test_data, y_test_data) = data['TRAIN'], data['TEST']
        # print(X_test_data.shape, X_train_data.shape)
        for classifi_obj in classifi_objs:
            config = {
                'length_signal': X_train_data.shape[-1],
                'name_model': name_model
            }
            
            model = get_model(**config)

            # print(name_model)
            config_train = setup_config_train(X_train_data, y_train_data, X_test_data, y_test_data, name_model)
            train_model(model, classifi_obj, **config_train)

            motion = 'VALENCE' if classifi_obj else 'AROUSAL'

            models.append((*experiment[:2], motion, model))

            info = (
                name_model,
                *experiment[:2], 
                motion,
                evaluate(model, name_model, classifi_obj, X_test_data, y_test_data)['Acc']*100
            )
            i += 1
            print(f'experiment {i}: {info}')
            evaluates.append(info)
            
            

experiment 1: ('rf', 'WT', 'YES', 'AROUSAL', 78.125)
experiment 2: ('rf', 'WT', 'YES', 'VALENCE', 76.953125)
experiment 3: ('rf', 'WT', 'NO', 'AROUSAL', 78.125)
experiment 4: ('rf', 'WT', 'NO', 'VALENCE', 76.5625)
experiment 5: ('rf', 'PE', 'NO', 'AROUSAL', 78.90625)
experiment 6: ('rf', 'PE', 'NO', 'VALENCE', 77.34375)
experiment 7: ('rf', 'NONE', 'YES', 'AROUSAL', 77.34375)
experiment 8: ('rf', 'NONE', 'YES', 'VALENCE', 68.75)
experiment 9: ('rf', 'NONE', 'NO', 'AROUSAL', 78.125)
experiment 10: ('rf', 'NONE', 'NO', 'VALENCE', 71.875)
experiment 11: ('svm', 'WT', 'YES', 'AROUSAL', 78.515625)
experiment 12: ('svm', 'WT', 'YES', 'VALENCE', 77.34375)
experiment 13: ('svm', 'WT', 'NO', 'AROUSAL', 78.515625)
experiment 14: ('svm', 'WT', 'NO', 'VALENCE', 77.34375)
experiment 15: ('svm', 'PE', 'NO', 'AROUSAL', 79.296875)
experiment 16: ('svm', 'PE', 'NO', 'VALENCE', 77.34375)
experiment 17: ('svm', 'NONE', 'YES', 'AROUSAL', 79.296875)
experiment 18: ('svm', 'NONE', 'YES', 'VALENCE', 77.34375

100%|██████████| 20/20 [00:30<00:00,  1.55s/it]


experiment 21: ('rnn', 'WT', 'YES', 'AROUSAL', 79.296875)


100%|██████████| 20/20 [00:30<00:00,  1.54s/it]


experiment 22: ('rnn', 'WT', 'YES', 'VALENCE', 76.953125)


...

experiment 38: ('cnn', 'NONE', 'YES', 'VALENCE', 75.390625)


100%|██████████| 20/20 [00:09<00:00,  2.16it/s]


experiment 39: ('cnn', 'NONE', 'NO', 'AROUSAL', 78.90625)


100%|██████████| 20/20 [00:09<00:00,  2.15it/s]

experiment 40: ('cnn', 'NONE', 'NO', 'VALENCE', 75.0)
